<img src="images/strathsdr_banner.png" align="left" >

# RFSoC OFDM Transceiver
----

<div class="alert alert-box alert-info">
Please use Jupyter Labs http://board_ip_address/lab for this notebook.
</div>

This notebook demonstrates the implementation of an Orthogonal Frequency Division Multiplexing (OFDM) transceiver on the RFSoC2x2 board. PYNQ is used to control the underlying modulation scheme of the OFDM sub-carriers and for visualisation of data at various stages in the transmit/receive chain, such as the received constellations.       

## Aims 
* To demonstrate a complete OFDM transceiver.
* Explain the various stages OFDM comprises.
* Provide an interactive and responsive means to inspect the data at each stage.

## Table of Contents
* [Introduction](#introduction)
    * [Hardware Setup](#hardware-setup)
    * [Software Setup](#software-setup)
* [Transmit](#transmit)
    * [Symbol Generation](#symbol-gen)
    * [Transmit Signal](#transmit-signal)
* [Receive](#creating-images)
    * [Channel Frequency Response](#channel-frequency-response)
    * [Constellation Plot](#constellation-plot)
* [Conclusion](#conclusion)

## References
* [Xilinx, Inc, "USP RF Data Converter: LogiCORE IP Product Guide", PG269, v2.3, June 2020](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_3/pg269-rf-data-converter.pdf)

## Revision History
* **v1.0** | 26/02/2021 | OFDM transceiver notebook

----

## Introduction <a class="anchor" id="introduction"></a>

The demonstrator is a complete OFDM transceiver. This notebook will explain each stage of the system with a combination of text, diagrams and live data capture. [Figure 1](#fig-1) below provides an overview of the system.

<a class="anchor" id="fig-1"></a>
<figure>
<img src="./images/system_overview.png" height='75%' width='75%'/>
    <figcaption><b>Figure 1: OFDM Demonstrator System Overview</b></figcaption>
</figure>

The OFDM system starts with generation of random data symbols from 1 of 10 possible modulation schemes (BPSK to 1024 QAM), based on input provided from PYNQ. In accordance with the procedure used in the IEEE 802.11a/g standard, the symbols are grouped into blocks of 48 for mapping to sub-carriers. The OFDM symbol consists of 48 data sub-carriers, 4 pilot sub-carriers and 12 null sub-carriers (including DC). The final OFDM symbol is created by performing a 64 point IFFT and addding a 16 sample Cyclic Prefix (CP). The transmitted signal consists of a continuous stream of OFDM symbols. At the very beginning of the data stream, the L-STF and L-LTF training symbols from the IEEE 802.11a/g standard are transmitted, to aid synchronisation and channel estimation tasks in the receiver. 

In the receiver, timing and frequency synchronisation are performed to acquire symbol timing and correct for any frequency offsets. Once timing is achieved, the FFT is performed to recover the underlying data symbols in each OFDM symbol. The L-LTF symbols are used to estimate the channel frequency response at each sub-carrier position, which subsequently allows the data to be equalised. Finally, the pilot symbols are used to correct for residual phase errors in the phase tracking stage. The recovered symbols are then passed into the PS for visualisation in PYNQ.

### Hardware Setup <a class="anchor" id="hardware-setup"></a>
This demonstration is intended to operate in the 2nd Nyquist Zone, at 2.4 GHz. Therefore, some analogue bandpass filters are required to prevent unwanted interference. 

The **Mini Circuits 15542 VBF-2435+ Coaxial Bandpass Filter** is suitable for this loopback demonstrator. Giving a passband of 2340 MHz to 2530 MHz.

Your RFSoC2x2 development board should be setup in single channel mode. There are four SMA interfaces on your board that are labelled DAC1, DAC2, ADC1, and ADC2. To setup your board for this demonstration, you can connect a loopback channel as shown in [Figure 2](#fig-2).

The default loopback configuration in this demonstration is connected as follows:
* DAC1 to ADC1

<a class="anchor" id="fig-2"></a>
<figure>
<img src='images/rfsoc2x2_setup.jpg' height='50%' width='50%'/>
    <figcaption><b>Figure 2: RFSoC2x2 development board setup in loopback mode.</b></figcaption>
</figure>

The loopback connection will be useful for running the OFDM demonstrator. **Do Not** attach an antenna to any SMA interfaces labelled DAC1 or DAC2.

<div class="alert alert-box alert-danger">
<b>Caution:</b>
    In this demonstration, we generate signals using the RFSoC development board. Your device should be setup in loopback mode. You should understand that the RFSoC platform can also transmit RF signals wirelessly. Remember that unlicensed wireless transmission of RF signals may be illegal in your geographical location. Radio signals may also interfere with nearby devices, such as pacemakers and emergency radio equipment. Note that it is also illegal to intercept and decode particular RF signals. If you are unsure, please seek professional support.
</div>

### Software Setup
The setup for the OFDM demonstration system is nearly complete. The majority of the libraries used by the demonstrator design are contained inside the RFSoC-OFDM software package. We only need to run a few code cells to initialise the environment.

In [ ]:
from rfsoc_ofdm.overlay import OfdmOverlay

ofdm = OfdmOverlay()

----

## Transmit <a class="anchor" id="transmit"></a>

### Symbol Generation <a class="anchor" id="symbol-gen"></a>
There are a total of 10 modulation schemes available to be transmitted. These are generated on the programmable logic and can be chosen between by updating the *ofdm_tx IP core's* **mod** register with a value from 0-9 over AXI4-Lite. [Figure 3](#fig-3) illustrates the IP core as a simplified block diagram.

<a class="anchor" id="fig-3"></a>
<figure>
<img src="./images/symbol_generation.png" height='45%' width='45%'/>
    <figcaption><b>Figure 3: Symbol generation block diagram.</b></figcaption>
</figure>


This drop down widget sends the value associated with each modulation scheme to the ofdm_tx core. Run the cell to use it.

In [ ]:
ofdm.modulation_type()

The output of the symbol generation block has been tapped off, allowing for the live symbols to be visualised in Jupyter Lab. Run the cell below and hit play on the chart to inspect the symbols generated on the programmable logic.

Try changing the modulation type using the dropdown widget above while the plot is updating.

In [ ]:
ofdm.plot_group('tx_sym', ['time-binary'], ofdm.ofdm_tx.get_tx_sym, fs=1e6)

### Transmit Signal <a class="anchor" id="transmit-signal"></a>

A further tap off point shows the transmit signal below. The symbols above have been mapped to sub carriers. An IFFT has then been performed and a cyclic prefix added. The stage tapped off is during the interpolation chain where the signal has been interpolated by a factor of 4, giving a sample rate of 4 MHz. 

Before running this plot, stop the one above if not already.

In [ ]:
ofdm.plot_group('tx_4M', ['time-binary'], ofdm.ofdm_tx.get_tx_4M, fs=4e6, y_range=[-0.3,0.3])

## Receive <a class="anchor" id="receive"></a>

### Channel Frequency Response <a class="anchor" id="channel-frequency-response"></a>

In OFDM, the channel frequency response reduces to a single complex value per sub-carrier. The below plot shows the magnitude response at each sub-carrier position, as measured in the channel estimation subsytem. As mentioned, there are a total of 64 sub-carriers and the sub-carrier index ranges from $k=-32$ to $k=31$ including DC. Notice, the magnitude is zero at the DC and null sub-carrier positions, since no data is transmitted on these sub-carriers.        

Run the cell below to get a frame from the programmable logic containing the channel estimation and plot the response:

In [ ]:
import plotly.graph_objects as go
import numpy as np

plot_ch_est = ofdm.ofdm_rx.get_channel_estimate()

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(-32,32), y=plot_ch_est, mode='lines'))
fig.data[0].line = dict(shape='hvh')
fig.layout.title = 'Estimated Channel Frequency Response'
fig.layout.xaxis.title = 'Subcarrier Index'
fig.layout.yaxis.title = 'Magnitude (mV)'
fig.show()

### Constellation Plot <a class="anchor" id="constellation-plot"></a>

Run the cell below to see the output of the OFDM receiver (ofdm_rx IP core) displayed as a constellation: 

In [ ]:
ofdm.plot_group(
  'rx_demod',             # Plot group's ID
  ['constellation'],      # List of plot types chosen from:
                          # ['time','time-binary','frequency','constellation']
  ofdm.ofdm_rx.get_demod, # Function to grab a buffer of samples
)

Some controls have been provided. Run the cell below and then try changing the modulation scheme. 

Also, you can change the transmit and receive center frequencies of the RF data converters. Remember that if analogue filters are being used, to keep the center frequency within their passband. 

In [ ]:
import ipywidgets as ipw
ipw.HBox([ofdm.modulation_type(), ofdm.carrier_frequency()])

## Conclusion <a class="anchor" id="conclusion"></a>
This notebook has demonstrated a live OFDM transceiver operating on an RFSoC. It has been shown how PYNQ can be leveraged to interact with various parts of the hardware design, offering control of the system and visualisation of data. 

* The various components comprising the OFDM transmitter and receiver have been introduced at a high level. 
    * Modulation symbols were inspected.
    * The transmitted signal was visualised.
    * A channel estimation, calculated in the OFDM receiver, was displayed.
    * The received and sychronised constellation was plotted.
* Interacted with a real-time RF system.
    * Changed modulation scheme.
    * Updated the Tx and Rx carrier frequencies in the RF data converters.

----
----